In [15]:
!pip -q install yfinance
!pip -q install matplotlib
!pip -q install cryptography
import yfinance as yf
import requests
import pandas as pd
import getpass
import base64, os
from datetime import datetime
from cryptography.fernet import Fernet

from cryptography.fernet import Fernet
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
import base64
import os

def key_from_password(password: str, salt: bytes):
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=base64.urlsafe_b64decode(salt),
        iterations=100000,
        backend=default_backend()
    )
    key = base64.urlsafe_b64encode(kdf.derive(password.encode()))
    return key

def decrypt_message(encrypted_message, password, salt):
    key = key_from_password(password, salt)
    fernet = Fernet(key)
    try:
        decrypted_message = fernet.decrypt(encrypted_message.encode()).decode()
        return decrypted_message
    except:
        return None

def fetch_bvb_data(ticker, startdate, enddate):
    start_timestamp = int(datetime.strptime(startdate, '%Y-%m-%d').timestamp())
    end_timestamp = int(datetime.strptime(enddate, '%Y-%m-%d').timestamp())
    url = f'https://wapi.bvb.ro/api/history?symbol={ticker}&dt=DAILY&p=day&ajust=0&from={start_timestamp}&to={end_timestamp}'
    referrer_url = decrypt_message(
        'gAAAAABlq-CfDvJGetCCbdadT9aEdRwD9AjQNL9CcYh8aINR_uiBOOrD_58SDsTN9pO0IE4twO2ohut3SMEHpFRSfskGWv6N048H3y1yKQWWhyb-kpFPuLhFA6UkkTvLcKduTBwxXFG4m8LeP-RPkYcxi9TrHjX3uTkrFcBKIyA54XtgWcMvLr22XDeMaWLCg2bN85W5UadJ', 
        password,
        'RYGlNyoOQupNEt4l0f8IdQ=='
        )
    if not referrer_url:
        print("Incorrect password, cannot decrypt the URL.")
        return pd.DataFrame()
    headers = {'Referer': referrer_url}
    response = requests.get(url, headers=headers)
    del referrer_url
    del headers
    if response.status_code != 200:
        return pd.DataFrame()  # Return an empty DataFrame in case of unsuccessful response
    data = response.json()
    if data.get('s') != 'ok':
        return pd.DataFrame()  # Return an empty DataFrame if status is not 'ok'
    return pd.DataFrame({
        'Date': [datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d') for ts in data['t']],
        'Open': data['o'],
        'High': data['h'],
        'Low': data['l'],
        'Close': data['c'],
        'Volume': data['v']
    })

password = getpass.getpass("Enter your BVB fetcher password: ")
ticker = 'EL'
startdate = '2016-01-01'
enddate = '2021-01-01'
df = fetch_bvb_data(ticker, startdate, enddate)
print(df)

del password

TypeError: salt must be bytes